In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#라이브러리와 app 데이터 세트 로딩
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.offline import iplot

# 화면크기 설정
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# 파일 읽어오기

heart=pd.read_csv("../input/heart-disease-uci/heart.csv")
print('Testing data shape:', heart.shape)
heart.head()

In [ ]:
heart.columns

# -columns 값 정의 -
1. age : 연령 
2. sex : 성(1=남자, 0=여자)
3. cp : 가슴 통증 유형 (1-4)
4. trestbps : 휴식 혈압 
5. chol : mg/dl의 혈청 콜레스트랄
6. fbs : 금식 혈당 > 120 mg/dl 
7. restecg : 휴식 심전도 결과 (값 0,1,2) 
8. thalach : 최대 심박수 달성
9. exang : 운동 유도 협심증 
10. oldpeak : 올드피크 = ST 우울증은 휴식에 상대적인 운동에 의해 유도
11. slope : 피크 연습 ST 세그먼트의 경사 
12. ca : 밀가루에 의해 착색 된 주요 선박 (0-3)의 수 
13. thal : thal: 3 = 정상; 6 = 고정 결함; 7 = 뒤집을 수 있는 결함 
14. target : 심장마비 발생유무 (1=발생, 0=미발생) 

In [ ]:
heart.info()
#모든 변수에 대해 결측치가 없음을 확인함 --> 결측치 처리 불필요함

In [ ]:
# 시각화를 했을 때의 가독성을 위해 column의 이름을 자세히 재구성한다.

heart.columns = ['age','sex','chest_pain','resting_blood_pressure','cholesterol',
                 'fasting_blood_sugar','rest_ecg','max_heart_rate','exercise_angina',
                 'st_depression','st_slope','num_major_vessels','thalassemia','target']

In [ ]:
heart.head()

In [ ]:
heart1 = heart.copy() # 숫자로 이루어진 파일 복사본

In [ ]:
#heart의 각 속성에 맞게 숫자에 해당하는 문자열을 대체한다.

heart.sex = heart.sex.map({0:'female',
                           1:'male'})

heart.chest_pain = heart.chest_pain.map({1:'angina pectoris',
                                         2:'atypical angina',
                                         3:'non-anginal pain',
                                         4:'SMI',
                                         0:'absent'})

heart.fasting_blood_sugar = heart.fasting_blood_sugar.map({0:'lower than 120mg/ml',
                                                           1:'greater than 120mg/ml'})

heart.exercise_angina = heart.exercise_angina.map({0:'no', 1:'yes'})

heart.st_slope = heart.st_slope.map({1:'upsloping',
                                     2:'horizontal', 
                                     3:'downsloping', 
                                     0:'absent'})

heart.thalassemia = heart.thalassemia.map({1:'normal', 
                                           2:'fixed defect', 
                                           3:'reversable defect', 
                                           0:'absent'})

heart.target = heart.target.map({0:'No Heart Disease',1:'Heart Disease'})

In [ ]:
heart.head()

In [ ]:
#심장질환 발병전체건수
sns.countplot(x="target", data=heart, hue="target")
plt.show()

# Age, Sex 열

In [ ]:
#성별에 따른 심장질환 발병건수
sns.countplot(x="sex", data=heart, hue="target")
plt.show()

In [ ]:
# 연령대에 따른 심장 질환 분포도
data = [heart['age']]
group = ['age'] 

# 'thalassemia' 히스토그램 
fig = ff.create_distplot(data, group_labels=group,bin_size=10,show_rug=False)
fig.update_layout(width=700, title_text='연령별 분포도')
fig.show()

- result 해석
- 연령 분포는 정규 분포에 가깝게 보임. 
- 이 데이터의 환자 연령은 29-77세로 특정 유형에 치우치지 않음.

In [ ]:
# 성별에 따른 나이와 심장질환의 상관관계 (막대그래프)

fig = px.bar(heart, x=heart['target'], y=heart['age'], color='sex', height=500, width=800)
fig.update_layout(title_text='성별에 따른 나이와 심장질환의 상관관계(막대그래프)')
fig.show()

- result 해석
- 심장병을 앓고 있는 사람과 심장병을 앓고 있지 않은 사람은 나이가 거의 비슷함.
- 심장병을 앓고 있는 남녀 환자의 수가 거의 같음.

In [ ]:
# 성별에 따른 나이와 심장질환의 상관관계 (박스플랏)

fig = px.box(heart, x="target", y="age", points="all", color='sex')
fig.update_layout(title_text='성별을 기반으로 한 나이 VS 심장질환')
fig.show()

- result 해석
- 심장질환이 없는 환자의 연령 중간값은 심장질환 환자보다 약간 높음.

# Chest Pain

In [ ]:
# 심장질환자와 비심장질환자의 흉통유형수

sns.countplot(x="chest_pain", hue="target", data=heart)

- result 해석
- 가장 흔한 흉통 유형은 "atypical agina"이고, "agina pectoris"가 다음이다. 
- 40명 정도의 환자가 가슴통증은 없지만 여전히 심장병을 앓고 있다는 것을 알 수 있기 때문에 가슴통증이 없다고 해서 환자가 심장병이 없다는 것을 보장할 수는 없다.

In [ ]:
# 남성/여성 환자의 흉통유형수

sns.countplot(x="chest_pain", hue="sex", data=heart)

- result 해석
- 가슴에 통증이 없는 남성 환자 > 여성 환자

# Resting Blood Pressure

In [ ]:
# 휴식기 혈압에 따른 심장 질환 분포도
data = [heart['resting_blood_pressure']]
group = ['resting_blood_pressure'] 

# 'thalassemia' 히스토그램 
fig = ff.create_distplot(data, group_labels=group,bin_size=10,show_rug=False)
fig.update_layout(width=700, title_text='휴식기 혈압별 분포도')
fig.show()

In [ ]:
# 휴식기 혈압과 심장질환의 상관관계 (막대그래프)

sns.barplot(x="target", y='resting_blood_pressure',data = heart)
plt.show()

In [ ]:
# 휴식기 혈압과 심장질환의 상관관계(박스플랏)

fig = px.box(heart, x="target", y="resting_blood_pressure", points="all", color='sex')
fig.update_layout(title_text='휴식기 혈압과 심장질환의 상관관계')
fig.show()

- result 해석
- 휴식기 혈압 분포는 정상 분포에 가깝다.
- 심장질환 환자와 비심장질환 환자의 휴식기 혈압은 비슷하다.
- 심장질환이 있는 중간 휴식기 혈압 = 남성(130) / 여성(130)
- 심장질환이 없는 중간 휴식기 혈압 = 남성(130) / 여성(140)

# Cholesterol

In [ ]:
# 콜레스테롤에 따른 심장 질환 분포도
data = [heart['cholesterol']]
group = ['cholesterol'] 

# 'thalassemia' 히스토그램 
fig = ff.create_distplot(data, group_labels=group,bin_size=10,show_rug=False)
fig.update_layout(width=700, title_text='콜레스테롤별 분포도')
fig.show()

In [ ]:
# 콜레스테롤과 심장질환의 상관관계 (막대그래프)

sns.barplot(x="target", y='cholesterol',data=heart)
plt.title('cholesterol vs target')
plt.show()

In [ ]:
# 성별을 기반으로 한 콜레스테롤과 심장질환의 상관관계 (박스플랏)

fig = px.box(heart, x="target", y="cholesterol", points="all", color='sex')
fig.update_layout(title_text='성별을 기반으로 한 콜레스테롤과 심장질환의 상관관계')
fig.show()

In [ ]:
# 세그먼트 경사를 기반으로 한 콜레스테롤과 심장질환의 상관관계 (박스플랏)

fig = px.box(heart,x="target",y="cholesterol",
             points="all",color='st_slope')
fig.update_layout(title_text='세그먼트 경사를 기반으로 한 콜레스테롤과 심장질환의 상관관계 ')
fig.show()

- result 해석
- 혈청 콜레스테롤의 분포는 정상 분포에 가깝고 꼬리가 길다.
- 심장질환 환자와 비심장질환 환자의 휴식기 혈압은 거의 같다.
- 심장질환이 있는 중간 콜레스테롤 = 남성(228) / 여성(249)
- 심장질환이 없는 중간 콜레스테롤 = 남성(247.5) / 여성(265.5)
- 세그먼트 경사는 차이가 별로 없음.

# Fasting Blood sugar

In [ ]:
# 공복 혈당이 120mg/dl 이상, 120mg/dl 미만인 남녀 환자의 수

sns.countplot(hue='fasting_blood_sugar',x ='sex',data = heart)
plt.title('fasting blood sugar (120mg/dl) by sex')
plt.show()

In [ ]:
# 공복 혈당이 120mg/dl 이상, 120mg/dl 미만인 심장질환 환자, 비심장질환 환자의 수

sns.countplot(hue='fasting_blood_sugar',x ='target',data = heart)
plt.title('fasting blood sugar (120mg/dl) by patient')
plt.show()

- result 해석
- 공복 혈당이 120mg/dl 미만인 환자는 심장질환을 앓을 가능성이 높음
- 여성 환자에 비해 120 mg/dl 이상을 먹는 남성 환자가 더 많다.

# ECG

In [ ]:
# 휴식기 심전도 결과 (values 0,1,2) by disease)

sns.countplot(x='rest_ecg', hue ='target', data = heart)
plt.title('Resting electrocardiographic Results by disease')
plt.show()

In [ ]:
# 휴식기 심전도 결과 (values 0,1,2) by sex)

sns.countplot(x='rest_ecg', hue ='sex', data = heart)
plt.title('Resting electrocardiographic Results by sex')
plt.show()

- result 해석
- 휴식기 심전도가 1인, 즉 심장질환을 앓고 있는 환자가 많다. 
- rest_computer(2)는 매우 적다.
- rest_hyp(0), rest_hyp(1)은 남성 환자에게 더 많이 발견된다.

# Thalassemia

In [ ]:
# 심장질환 기반 Thalassemia 유형 (막대그래프)

sns.countplot(hue='thalassemia',x ='target',data = heart)
plt.title('Thalassmia')
plt.show()

In [ ]:
# 가슴통증 기반 Thalassemia 유형 

plt.figure(figsize=(10,5))
sns.countplot(x="chest_pain", hue="thalassemia", data=heart)

In [ ]:
# 세그먼트 경사 기반 Thalassemia 유형

plt.figure(figsize=(10,5))
sns.countplot(x="st_slope", hue="thalassemia", data=heart)

In [ ]:
# 성별 기반 Thalassemia 유형

sns.countplot(hue='thalassemia',x ='sex',data =heart)
plt.show()

- result 해석

- 심장병의 유무와 무관하게 대부분의 환자들이 Thalassemia를 앓음.
- fixed defect Thalassemia는 심장질환 환자에게서 더 흔하며, reversable defect Thalassemia는 심장질환이 없는 환자에게서 더 흔하다.
- 모든 유형의 흉통에서 fixed defect Thalassemia 환자가 더 많은 반면 흉통이 없는 환자와 reversable defect Thalassemia는 매우 일반적이다.
- fixed defect Thalassemia는 남성과 여성 환자 사이에서 매우 흔하지만 여성 환자에 비해 reversable defect Thalassemia 남성 환자가 매우 많다.
- horizontal slope 환자 중 fixed defect Thalassemia는 다른 환자보다 더 일반적인 반면, upsloping 환자의 경우 reversable defect Thalassemia 환자 수가 더 많다.

# feature engineering
- feature들에 대해 TATGET값 분포도 비교 시각화

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import scikitplot as skplt
from sklearn.metrics import accuracy_score

In [ ]:
dummy1 = pd.get_dummies(heart1.chest_pain)
dummy2 = pd.get_dummies(heart1.thalassemia)
dummy3 = pd.get_dummies(heart1.rest_ecg)
dummy4 = pd.get_dummies(heart1.st_slope)
dummy5 = pd.get_dummies(heart1.num_major_vessels)
merge = pd.concat([heart1,dummy1,dummy2,dummy3,dummy4,dummy5],axis='columns')

I have created dummy variables for categorical data to increase the accuracy.

In [ ]:
final = merge.drop(['chest_pain','thalassemia',
                    'rest_ecg','st_slope','num_major_vessels'],axis = 1)
final.head()

In [ ]:
x = final.drop(['target'], axis = 1)
y = final['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.20, random_state = 5)

In [ ]:
feature_scaler = MinMaxScaler()
x_train = feature_scaler.fit_transform(x_train)
x_test = feature_scaler.transform(x_test)

In [ ]:
accuracy = []

In [ ]:
# 정확도는 리스트로 설정

# Decision Trees

In [ ]:
criterion = ['gini','entropy']
splitter = ['best','random']

Tree = DecisionTreeClassifier()

parameters = {'criterion': ['gini','entropy']}

tree_classifier = GridSearchCV(Tree, parameters, scoring='neg_mean_squared_error' ,cv =5)
tree_classifier.fit(x_train, y_train)
tree_classifier.best_params_

In [ ]:
model2 = DecisionTreeClassifier(criterion = 'gini')
model2.fit(x_train,y_train)
accuracy2 = model2.score(x_test,y_test)
accuracy.append(accuracy2)
print('Decision Tree Accuracy -->',((accuracy2)*100))

In [ ]:
pred2 = model2.predict(x_test)
matrix2 = (y_test,pred2)
skplt.metrics.plot_confusion_matrix(y_test ,pred2 ,figsize=(10,5))

- result 해석
- 이 결과로 10개의 값에 오류가 있음을 보여주고, 다른 값은 모두 올바르게 예측되었음을 보여줌.

# K-Nearest Neighbors

In [ ]:
Krange = range(1,20)
scores = {}
scores_list = []
for k in Krange:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(x_train,y_train)
    y_pred = knn.predict(x_test)
    scores[k] = metrics.accuracy_score(y_test,y_pred)
    scores_list.append(metrics.accuracy_score(y_test,y_pred))
    
plt.plot(Krange,scores_list)
plt.xlabel("Value of K")
plt.ylabel("Accuracy")

In [ ]:
model8 = KNeighborsClassifier(n_neighbors = 5)
model8.fit(x_train,y_train)
accuracy8 = model8.score(x_test,y_test)
accuracy.append(accuracy8)
print('Gradient Boosting Classifier Accuracy -->',((accuracy8)*100))

In [ ]:
pred8 = model8.predict(x_test)
matrix8 = (y_test,pred8)
skplt.metrics.plot_confusion_matrix(y_test ,pred8 ,figsize=(10,5))

- result 해석
- 이 결과로 5개의 값에 오류가 있음을 보여주고, 다른 값은 모두 올바르게 예측되었음을 보여줌.